In [33]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
p = "DigitDataset/"
dir = os.listdir(p)
df = pd.DataFrame(columns =['path','label'])

path_to_data = []
class_name = []
for el in dir:
    if os.path.isdir(p+el):
        DIR = os.listdir(p+el)
        for label in DIR:
            path_to_data.append(p+el+"/"+label)
            class_name.append(el)
            
s = set(class_name)

df = pd.DataFrame(data={'path':path_to_data, 'label':class_name})

sh = df.shape[0]
array_a = np.random.permutation(range(sh))
id_train = round(0.8*sh)
id_test = round(0.1*sh)
id_val = round(0.1*sh)

train_idx = array_a[0:id_train]
df_train = df.iloc[train_idx,:]
val_idx = array_a[id_train+1:id_train+id_val]
df_val = df.iloc[val_idx,:]
test_idx = array_a[id_train+id_val+1:-1]
df_test = df.iloc[test_idx,:]





In [34]:
from torch.utils.data import Dataset
s = list(s)

class imgDataset(Dataset):
    def __init__(self,df,s=s, transform=None):
        self.df = df
        self.class_names = s
        if transform is None:
            self.should_transform = False
        else:
            self.transform = transform
            self.should_transform = True
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        label = self.df.iloc[idx, 1]
        label = int(label)
        #print(self.class_names)
        #print(type(self.class_names))
       # label = self.class_names.index(label)
        
       
        image = plt.imread(self.df.iloc[idx,0])
    
       
        if self.should_transform:
            image = self.transform(image)          
        else:
            image = image
            
        return {"image": image, "label": label} 
  

from torchvision import transforms 

    
data_transform_list = [transforms.ToPILImage(),
                       transforms.ToTensor()]

data_transf_val = [transforms.ToPILImage(),
                       transforms.ToTensor()]

transform = transforms.Compose(data_transform_list)

transform_val = transforms.Compose(data_transf_val)
dataset = imgDataset(df_train,transform=transform)

dataset_val = imgDataset(df_val,transform=transform_val)


In [42]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
import torch.nn as nn
import torch.nn.functional as F

#==========================
# добавить слой нормировки по батчу
# добавить еще один слой свертки и сравнить результаты с исходной реализацией
# kernel_size сдедать равным 3

class ff_net(nn.Module):
    def __init__(self):
        super(ff_net, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=3,              
                stride=1,                   
                padding=1,                  
            ),                              
            nn.ReLU(), 
            
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 3, 1, 1),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, 3, 1, 0),
            nn.ReLU())
                                   
                                   
        self.out = nn.Linear(32 * 5 * 5, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output 
    
model = ff_net().to('cuda')

 
criterion = nn.CrossEntropyLoss()


cuda


In [43]:
print(model)

ff_net(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (out): Linear(in_features=800, out_features=10, bias=True)
)


In [44]:
from torch.utils.data import DataLoader
data_load = DataLoader(dataset,batch_size=32, shuffle=True, pin_memory=True) 


In [45]:
import torch.optim as optim

NUM_EPOCHS = 2
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(NUM_EPOCHS):
    test_error_count = 0.0
    for i, batch in enumerate(data_load):
    
        images = batch['image'].to('cuda')
        labels = batch['label'].to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        test_error_count += float(torch.sum((labels == outputs.argmax(1))))
        
    # =============================================================================
    #     добавить код тестирования на валидации в процессе обучения
    #     
    #        with torch.no_grad 
    #         
    #         
    #         
    # =============================================================================

    test_accuracy = 1.0 - float(test_error_count) / float(len(dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy  

0: 0.875500
1: 0.806000
